**From Detailed Cleaning/Visualization (Python) by Alan (AJ) Pryor, Jr.**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

**Import library and data**

In [ ]:
import 

In [ ]:
limit_rows = 100000

train = pd.read_csv('../input/train_ver2.csv',nrows=limit_rows,
                   dtype={'sexo':str,
                         'ind_nuevo':str,
                         'ult_fec_cli_lt':str,
                         'index':str})
train.head()

In [ ]:
unique_ids = pd.Series(train['ncodpers'].unique())
limit_people = 1.2e4
unique_id = unique_ids.sample(n=limit_people)
train = train[train.ncodpers.isin(unique_id)]
train.describe()

String to Datetime

In [ ]:
train["fecha_dato"] = pd.to_datetime(train["fecha_dato"],format="%Y-%m-%d")
train["fecha_alta"] = pd.to_datetime(train["fecha_alta"],format="%Y-%m-%d")
train["fecha_dato"].unique()

In [ ]:
train['month'] = pd.DatetimeIndex(train['fecha_dato']).month
train['age']   = pd.to_numeric(train['age'], errors='coerce')

train.isnull().any()

**Data Cleaning**

In [ ]:
with sns.plotting_context('notebook', font_scale=1.5):
    sns.set_style('whitegrid')
    sns.distplot(train['age'].dropna(), bins=80, kde=False, color='tomato')
    sns.plt.title('Age Distribution')
    plt.ylabel('Count')

replace the age below 18 and above 100 with the close one

In [ ]:
train.loc[train.age <18, 'age'] = train.loc[(train.age>=18) & (train.age<=30), 'age'].mean(skipna=True)
train.loc[train.age >100, 'age'] = train.loc[(train.age <=100) & (train.age >=30), 'age'].mean(skipna=True)
train['age'].fillna(train['age'].mean(), inplace=True)
train['age'] = train['age'].astype(int)

In [ ]:
with sns.plotting_context('notebook', font_scale=1.5):
    sns.set_style('whitegrid')
    sns.distplot(train['age'].dropna(),
                 bins=80,
                 kde=False,
                 color='b')
    sns.plt.title('Age Distribution')
    plt.ylabel('Count')
    plt.xlim((15,100))
                 
       

Next: ind_nuevo, which indicates whether a customer is new or not.Count missint values.

In [ ]:
train['ind_nuevo'].isnull().sum()

In [ ]:
months_active = train.loc[train['ind_nuevo'].isnull(),:].groupby('ncodpers',sort=False).size()
months_active.max()

In [ ]:
train.loc[train['ind_nuevo'].isnull(), 'ind_nuevo'] = 1

In [ ]:
Now, antiguedad

In [ ]:
train.antiguedad = pd.to_numeric(train.antiguedad, errors='coerce')
np.sum(train['antiguedad'].isnull())

In [ ]:
train.loc[train['antiguedad'].isnull(), 'ind_nuevo'].describe()

In [ ]:
The same people which ind_nuevo is null above

In [ ]:
train['antiguedad'].describe()

In [ ]:
train.loc[train['antiguedad'].isnull(), 'antiguedad'] = train.antiguedad.min()
train.loc[train['antiguedad'] <0, 'antiguedad'] = 0
train.describe()

Some entries do not have the date the joined the company.Just give them something in the middle of the pack.

In [ ]:
dates = train.loc[:,'fecha_alta'].sort_values().reset_index()
median_date = int(np.median(dates.index.values))
median_date